In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# import numpy as np
# import pandas as pd
import librosa
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# import matplotlib.pyplot as plt


In [ ]:
# Load the Excel file into a pandas DataFrame
file_path = '/home/amit-talmale/Documents/GitHub/Articulation-Meter/Project Files/merged_excel_file.xlsx'  # Replace with your Excel file path
df = pd.read_excel(file_path)

In [ ]:
df.info()

In [ ]:
# change duration in sec. to min.

df['duration'] = df['duration'] / 60

In [ ]:
df.shape

In [ ]:
# treatment outliers by mean

columns = ['views', 'duration']

for i in columns:
  iqr = df[i].quantile(0.75)-df[i].quantile(0.30)
  df[i] = df[i].mask(df[i]>(df[i].quantile(0.75)+1.5*iqr), df[i].mean())

In [ ]:
#let's see the numerical column again after treating outliers

columns = ['views', 'duration']
n = 1
plt.figure(figsize=(18,12))

for i in columns:
  plt.subplot(3,3,n)
  n=n+1
  sns.boxplot(df[i])
  plt.title(i)
  plt.tight_layout()

In [ ]:
import numpy as np
import re

# Define a function to convert string representations of arrays to numpy arrays
def convert_to_array(mfccs_str):
    # Extract numerical values from the string using regular expressions
    values = re.findall(r'[-+]?\d*\.\d+|\d+', mfccs_str)
    # Ensure that the number of values is divisible by 13 (assuming 13 MFCCs features)
    num_values = len(values)
    if num_values % 13 != 0:
        # Pad or truncate the values to make the number divisible by 13
        target_length = num_values + (13 - (num_values % 13))
        if num_values < target_length:
            values += ['0'] * (target_length - num_values)
        else:
            values = values[:target_length]
    # Convert the values to float and reshape into a numpy array
    mfccs_array = np.array(values, dtype=float).reshape(-1, 13)
    return mfccs_array

# Convert MFCCs strings to numpy arrays
df['MFCCs'] = df['MFCCs'].apply(convert_to_array)


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Flatten MFCCs arrays and add them as separate features
mfcc_columns = [f'mfcc_{i}' for i in range(df['MFCCs'].iloc[0].shape[1])]
for i, col in enumerate(mfcc_columns):
    df[col] = df['MFCCs'].apply(lambda x: x[:, i])

# Select relevant columns for clustering
selected_columns = ['duration', 'likes', 'views', 'Energy', 'Pitch', 'Speech Rate Variation', 'Articulation Rate', 'Frequency'] + mfcc_columns
data_for_clustering = df[selected_columns]

# Exclude non-numeric columns before scaling
numeric_columns = data_for_clustering.select_dtypes(include=['float64', 'int64']).columns
data_for_scaling = data_for_clustering[numeric_columns]

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_for_scaling)

# Perform clustering using KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(scaled_data)

# Add the cluster labels to the DataFrame
df['cluster'] = clusters

# Print the cluster centers
print("Cluster Centers:")
print(scaler.inverse_transform(kmeans.cluster_centers_))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Perform PCA for dimensionality reduction to 2D
pca = PCA(n_components=2)
reduced_data = pca.fit_transform(scaled_data)

# Plot the clusters
plt.figure(figsize=(10, 6))
for cluster in range(3):
    plt.scatter(reduced_data[clusters == cluster, 0], reduced_data[clusters == cluster, 1], label=f'Cluster {cluster}')
plt.title('Clustering Visualization using PCA')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.show()


In [ ]:

#Example for understand the clustring

# Cluster 0:

# Duration: Approximately 3.75 units
# Likes: Approximately 126,860
# Views: Approximately 2,163,990
# Energy: Approximately 79,282,684
# Pitch: Approximately 3,999.72
# Speech Rate Variation: Approximately 4.92%
# Articulation Rate: Approximately 0.02%
# Frequency: Approximately 0.
# Cluster 1:

# Duration: Approximately 3.89 units
# Likes: Approximately 26,051
# Views: Approximately 882,478
# Energy: Approximately 82,660,148
# Pitch: Approximately 3,999.74
# Speech Rate Variation: Approximately -0.19%
# Articulation Rate: Approximately 0.01%
# Frequency: Approximately 0.004%
# Cluster 2:

# Duration: Approximately 3.54 units
# Likes: Approximately 35,764
# Views: Approximately 1,000,438
# Energy: Approximately 74,820,612
# Pitch: Approximately 3,999.43
# Speech Rate Variation: Approximately 0.28%
# Articulation Rate: Approximately 0.01%
# Frequency: Approximately 0.006%